In [21]:
from TauLidarCommon.frame import FrameType, Frame
from TauLidarCamera.camera import Camera
from TauLidarCamera.constants import VALUE_20MHZ
from TauLidarCommon.color import ColorMode

In [54]:
camera = Camera.open()

Looking for connected Tau LiDAR Camera hardware ...


In [26]:
cameraInfo = camera.info()

print("\nToF camera opened successfully:")
print("    model:      %s" % cameraInfo.model)
print("    firmware:   %s" % cameraInfo.firmware)
print("    uid:        %s" % cameraInfo.uid)
print("    resolution: %s" % cameraInfo.resolution)
print("    port:       %s" % cameraInfo.port)


ToF camera opened successfully:
    model:      4.0
    firmware:   3.3
    uid:        9503B7
    resolution: 160x60
    port:       /dev/ttyACM0


In [27]:
camera.setModulationChannel(0) ## autoChannelEnabled: 0, channel: 0

In [28]:
camera.setIntegrationTime3d(0, 800)  ## set integration time 0: 1000
camera.setMinimalAmplitude(0, 60)  ## set minimal amplitude 0: 80
camera.setRange(0, 4500) ## points in the distance range to be colored
camera.setIntegrationTimeGrayscale(15000)  ## set integration time grayscale: 8000, needed when requiring FrameType.DISTANCE_GRAYSCALE

In [29]:
frame = camera.readFrame(FrameType.DISTANCE)
frame = camera.readFrame(FrameType.DISTANCE_GRAYSCALE)
frame = camera.readFrame(FrameType.DISTANCE_AMPLITUDE)

In [30]:
frame = camera.readFrame(FrameType.DISTANCE_AMPLITUDE)

In [56]:
import cv2
import numpy as np
import time
from ultralytics import YOLO
import math

TESTING WITH YOLOV5s

In [32]:
# Using the YOLOv8s model
model = YOLO("/home/g05/yolov5/yolov5s.pt")

# If you have the custom model. Provide the absolute path to run the custom model
# path = "C://Users//jaura//Downloads//runs//detect//yolov8s_custom11//weights//best.pt"
# model = YOLO(path)

PRO TIP 💡 Replace 'model=/home/g05/yolov5/yolov5s.pt' with new 'model=/home/g05/yolov5/yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



In [ ]:
import random
# Start capturing video
try:
    while True:
        # Read a frame from the camera
        frame = camera.readFrame(FrameType.DISTANCE_AMPLITUDE)
        mat_amplitude = np.frombuffer(frame.data_amplitude, dtype=np.float32, count=-1, offset=0).reshape(frame.height, frame.width)
        frame = camera.readFrame(FrameType.DISTANCE)
        mat_distance = np.frombuffer(frame.data_depth, dtype=np.float32, count=-1, offset=0).reshape(frame.height, frame.width)

        # Convert to 8-bit for display
        # Normalize the image for better visualization
        cv2.normalize(mat_amplitude, mat_amplitude, 0, 255, cv2.NORM_MINMAX)
        mat_amplitude = mat_amplitude.astype(np.uint8)

        # Apply a colormap to the image
        colored_image = cv2.applyColorMap(mat_amplitude, cv2.COLORMAP_JET)

        # Perform object detection
        results = model.predict(source=colored_image, conf=0.5)  # Confidence threshold

        # Define a list of colors for different classes
        colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for _ in range(len(model.names))]

        for result in results:
            boxes = result.boxes  # Bounding box predictions
            for box in boxes:
                # Convert tensor to a NumPy array and extract coordinates
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)

                # Calculate center point
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2) 

                print(f"Center: ({center_x}, {center_y}) | Frame: {frame.width}x{frame.height} | Points_3D Length: {len(frame.points_3d)}")
                
                # Resize yolov5s coordinates to LiDAR Camera resolusion
                scale_x = frame.width / 640
                scale_y = frame.height / 256
                center_x = int(center_x * scale_x)
                center_y = int(center_y * scale_y)

                if center_x >= frame.width or center_y >= frame.height:
                    print(f"Skipping detection at ({center_x}, {center_y}) - Out of bounds")
                    continue

                # Read depth data from camera
                distance = mat_distance[center_y, center_x]

                #Get center point 3D loctaion
                index = min(center_y * frame.width + center_x, len(frame.points_3d) - 1)

                # if index >= len(frame.points_3d):
                #     print(f"Skipping index {index} - Out of points_3d range")
                #     continue

                point_data = frame.points_3d[index]
                x, y, z = point_data[:3]

                # Extract confidence and class ID
                conf = box.conf[0].item()  # Confidence score
                cls = int(box.cls[0].item())  # Class ID

                # Generate the label
                label = f"{model.names[cls]}: {conf:.2f}"
                # distance_label = f"{z:.1f} m"
                position_label = f"X:{x:.1f}m Y:{y:.1f}m Z:{z:.1f}m"
                angle = math.degrees(math.atan2(x, z))
                angle_label = f"Angle: {angle:.1f}degree"

                # Get a unique color for the class
                box_color = colors[cls % len(colors)]
                text_color = (255, 255, 255)  # Always use white text for better visibility

                # Create a copy of the image for overlay (for transparency effect)
                overlay = colored_image.copy()

                # Draw the box around the detected object
                cv2.rectangle(colored_image, (x1, y1), (x2, y2), box_color, 2)

                # Determine font scale based on object size
                font_scale = max(0.3, min((x2 - x1) / 300, 0.6))  # Font size auto adjusts

                # Calculate text sizes dynamically
                (text_w, text_h), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, 1)
                # (text_w_dist, text_h_dist), _ = cv2.getTextSize(distance_label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, 1)
                (text_w_pos, text_h_pos), _ = cv2.getTextSize(position_label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, 1)
                (text_w_angle, text_h_angle), _ = cv2.getTextSize(angle_label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, 1)

                # Define padding and positions
                padding_x = 5
                padding_y = 5
                text_x = x1
                text_y = max(y2 - 5, text_h + padding_y)  # Prevents text from going out of image

                # Draw transparent background for labels
                def draw_transparent_rect(img, x, y, w, h, color, alpha=0.5):
                    sub_img = img[y:y+h, x:x+w]
                    bg = np.full(sub_img.shape, color, dtype=np.uint8)
                    cv2.addWeighted(bg, alpha, sub_img, 1 - alpha, 0, sub_img)

                # Draw classification label
                draw_transparent_rect(overlay, text_x - padding_x, text_y - text_h - padding_y, text_w + padding_x * 2, text_h + padding_y * 2, box_color, alpha=0.5)
                cv2.putText(overlay, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, 1)

                # # Draw distance label
                # text_y_distance = text_y - text_h - 10
                # draw_transparent_rect(overlay, text_x - padding_x, text_y_distance - text_h_dist - padding_y, text_w_dist + padding_x * 2, text_h_dist + padding_y * 2, box_color, alpha=0.5)
                # cv2.putText(overlay, distance_label, (text_x, text_y_distance), cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, 1)

                # Draw position label
                text_y_position = text_y - text_h - 10
                draw_transparent_rect(overlay, text_x - padding_x, text_y_position - text_h_pos - padding_y, text_w_pos + padding_x * 2, text_h_pos + padding_y * 2, box_color, alpha=0.5)
                cv2.putText(overlay, position_label, (text_x, text_y_position), cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, 1)

                # Draw angle label
                text_y_angle = text_y_position - text_h_pos - 10
                draw_transparent_rect(overlay, text_x - padding_x, text_y_angle - text_h_angle - padding_y, text_w_angle + padding_x * 2, text_h_angle + padding_y * 2, box_color, alpha=0.5)
                cv2.putText(overlay, angle_label, (text_x, text_y_angle), cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, 1)

                # Merge the transparent overlay with the original image
                cv2.addWeighted(overlay, 0.8, colored_image, 0.2, 0, colored_image)





        # Enlarge the image while preserving aspect ratio
        target_width = 1200  # Desired width
        target_height = 1000  # Desired height
        resized_image = cv2.resize(colored_image, (target_width, target_height), interpolation=cv2.INTER_LINEAR)

        # Display the colored image with bounding boxes
        cv2.imshow('Object Detection on Depth Map', resized_image)
        cv2.waitKey(1)
        time.sleep(0.01)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("Program stopped by user")

finally:
    # Release resources and close the window
    cv2.destroyAllWindows()


0: 256x640 (no detections), 681.7ms
Speed: 5.1ms preprocess, 681.7ms inference, 4.0ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 638.2ms
Speed: 3.5ms preprocess, 638.2ms inference, 2.4ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 1 clock, 638.8ms
Speed: 6.7ms preprocess, 638.8ms inference, 2.3ms postprocess per image at shape (1, 3, 256, 640)
Center: (82, 29) | Frame: 160x60 | Points_3D Length: 5335

0: 256x640 (no detections), 842.4ms
Speed: 3.3ms preprocess, 842.4ms inference, 3.8ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 793.6ms
Speed: 5.7ms preprocess, 793.6ms inference, 2.2ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 691.0ms
Speed: 3.3ms preprocess, 691.0ms inference, 3.8ms postprocess per image at shape (1, 3, 256, 640)

0: 256x640 (no detections), 682.4ms
Speed: 3.4ms preprocess, 682.4ms inference, 3.1ms postprocess per image at shape (1, 3, 256, 640)

0: 2